In [1]:
dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

- support(A->C) = support(A+C) [aka 'support'], range: [0, 1]

- confidence(A->C) = support(A+C) / support(A), range: [0, 1]

- lift(A->C) = confidence(A->C) / support(C), range: [0, inf]

- leverage(A->C) = support(A->C) - support(A)*support(C),
range: [-1, 1]

- conviction = [1 - support(C)] / [1 - confidence(A->C)],
range: [0, inf]

In [2]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder

In [3]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,Apple,Corn,Dill,Eggs,Ice cream,Kidney Beans,Milk,Nutmeg,Onion,Unicorn,Yogurt
0,False,False,False,True,False,True,True,True,True,False,True
1,False,False,True,True,False,True,False,True,True,False,True
2,True,False,False,True,False,True,True,False,False,False,False
3,False,True,False,False,False,True,True,False,False,True,True
4,False,True,False,True,True,True,False,False,True,False,False


In [4]:
from mlxtend.frequent_patterns import apriori

In [5]:
apriori(df, min_support=0.6)

,support,itemsets
0,0.8,(3)
1,1.0,(5)
2,0.6,(6)
3,0.6,(8)
4,0.6,(10)
5,0.8,"(3, 5)"
6,0.6,"(8, 3)"
7,0.6,"(5, 6)"
8,0.6,"(8, 5)"
9,0.6,"(10, 5)"


In [6]:
apriori(df, min_support=0.6, use_colnames=True)

,support,itemsets
0,0.8,(Eggs)
1,1.0,(Kidney Beans)
2,0.6,(Milk)
3,0.6,(Onion)
4,0.6,(Yogurt)
5,0.8,"(Eggs, Kidney Beans)"
6,0.6,"(Eggs, Onion)"
7,0.6,"(Milk, Kidney Beans)"
8,0.6,"(Onion, Kidney Beans)"
9,0.6,"(Yogurt, Kidney Beans)"


In [7]:
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.8,(Eggs),1
1,1.0,(Kidney Beans),1
2,0.6,(Milk),1
3,0.6,(Onion),1
4,0.6,(Yogurt),1
5,0.8,"(Eggs, Kidney Beans)",2
6,0.6,"(Eggs, Onion)",2
7,0.6,"(Milk, Kidney Beans)",2
8,0.6,"(Onion, Kidney Beans)",2
9,0.6,"(Yogurt, Kidney Beans)",2


In [8]:
frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.8) ]

,support,itemsets,length
5,0.8,"(Eggs, Kidney Beans)",2


In [9]:
frequent_itemsets[ frequent_itemsets['itemsets'] == {'Onion', 'Eggs'} ]

,support,itemsets,length
6,0.6,"(Eggs, Onion)",2


In [10]:
from mlxtend.frequent_patterns import association_rules

In [11]:
association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Eggs),(Kidney Beans),0.8,1.0,0.8,1.00,1.00,0.00,inf
1,(Kidney Beans),(Eggs),1.0,0.8,0.8,0.80,1.00,0.00,1.000000
2,(Eggs),(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.600000
3,(Onion),(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf
4,(Milk),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf
5,(Onion),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf
6,(Yogurt),(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf
7,"(Eggs, Onion)",(Kidney Beans),0.6,1.0,0.6,1.00,1.00,0.00,inf
8,"(Eggs, Kidney Beans)",(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.600000
9,"(Onion, Kidney Beans)",(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf


In [12]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.2)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Eggs),(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.600000
1,(Onion),(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf
2,"(Eggs, Kidney Beans)",(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.600000
3,"(Onion, Kidney Beans)",(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf
4,(Eggs),"(Onion, Kidney Beans)",0.8,0.6,0.6,0.75,1.25,0.12,1.600000
5,(Onion),"(Eggs, Kidney Beans)",0.6,0.8,0.6,1.00,1.25,0.12,inf


In [13]:
rules["antecedant_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedant_len
0,(Eggs),(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.600000,1
1,(Onion),(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf,1
2,"(Eggs, Kidney Beans)",(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.600000,2
3,"(Onion, Kidney Beans)",(Eggs),0.6,0.8,0.6,1.00,1.25,0.12,inf,2
4,(Eggs),"(Onion, Kidney Beans)",0.8,0.6,0.6,0.75,1.25,0.12,1.600000,1
5,(Onion),"(Eggs, Kidney Beans)",0.6,0.8,0.6,1.00,1.25,0.12,inf,1


In [14]:
rules[rules['antecedents'] == {'Eggs', 'Kidney Beans'}]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedant_len
2,"(Eggs, Kidney Beans)",(Onion),0.8,0.6,0.6,0.75,1.25,0.12,1.6,2


Invoice, Product, Order 개수로 이루어진 Table을 전처리 후 market basket analysis 수행

In [15]:
df = pd.read_excel('mba.xlsx')

In [16]:
df

,Invoice,Product,Orders,Sales value
0,131506,Product 20,1,40
1,131506,Product 21,1,80
2,131507,Product 11,1,80
3,131508,Product 19,1,32
4,131509,Product 31,1,9
5,131510,Product 11,1,80
6,131510,Product 20,2,40
7,131510,Product 20,1,40
8,131519,Product 11,1,80
9,131541,Product 11,1,80


In [17]:
df = df[['Invoice', 'Product', 'Orders']]

In [18]:
df.groupby(['Invoice','Product'],as_index = False)['Orders'].sum()

,Invoice,Product,Orders
0,131506,Product 20,1
1,131506,Product 21,1
2,131507,Product 11,1
3,131508,Product 19,1
4,131509,Product 31,1
5,131510,Product 11,1
6,131510,Product 20,3
7,131519,Product 11,1
8,131541,Product 11,1
9,131541,Product 20,1


In [19]:
df1 = df.groupby(['Invoice','Product'],as_index = False)['Orders'].sum().pivot('Invoice','Product').fillna(0)

In [21]:
df1

Orders                                                         \
Product Product 10 Product 11 Product 12 Product 13 Product 14 Product 15   
Invoice                                                                     
131506         0.0        0.0        0.0        0.0        0.0        0.0   
131507         0.0        1.0        0.0        0.0        0.0        0.0   
131508         0.0        0.0        0.0        0.0        0.0        0.0   
131509         0.0        0.0        0.0        0.0        0.0        0.0   
131510         0.0        1.0        0.0        0.0        0.0        0.0   
131519         0.0        1.0        0.0        0.0        0.0        0.0   
131541         0.0        1.0        0.0        0.0        0.0        0.0   
131542         0.0        1.0        0.0        0.0        0.0        0.0   
131543         0.0        0.0        0.0        0.0        0.0        0.0   
131559         0.0        0.0        0.0        0.0        0.0        0.0   
131560         0.0        0.0        0.0        0.0        0.0        1.0   
131584         0.0        1.0        0.0        0.0        0.0        0.0   
131597         0.0        0.0        0.0        1.0        0.0        0.0   
131598         0.0        0.0        1.0        0.0        0.0        0.0   
131599         0.0        1.0        0.0        0.0        0.0        0.0   
131619         0.0        2.0        0.0        0.0        0.0        0.0   
131620         0.0        1.0        0.0        0.0        0.0        0.0   
131650         0.0        0.0        0.0        1.0        0.0        0.0   
131651         0.0        0.0        0.0        0.0        0.0        0.0   
131652         0.0        1.0        0.0        0.0        0.0        0.0   
131678         0.0        0.0        0.0        0.0        0.0        0.0   
131679         0.0        0.0        0.0        0.0        0.0        0.0   
131680         0.0        0.0        0.0        0.0        1.0        0.0   
131706         0.0        0.0        0.0        0.0        0.0        0.0   
131707         0.0        0.0        0.0        0.0        0.0        0.0   
131708         0.0        0.0        0.0        0.0        0.0        0.0   
131709         0.0        1.0        0.0        0.0        0.0        0.0   
131710         0.0        0.0        0.0        0.0        0.0        0.0   
131711         0.0        0.0        0.0        0.0        0.0        0.0   
131712         0.0        0.0        0.0        0.0        0.0        0.0   
...            ...        ...        ...        ...        ...        ...   
1307220        0.0        0.0        0.0        0.0        0.0        0.0   
1307221        0.0        0.0        0.0        0.0        0.0        1.0   
1307222        0.0        0.0        0.0        0.0        0.0        0.0   
1307224        0.0        1.0        0.0        0.0        0.0        0.0   
1307261        0.0        0.0        0.0        0.0        1.0        0.0   
1307262        0.0        0.0        0.0        0.0        0.0        0.0   
1307286        0.0        1.0        0.0        0.0        0.0        0.0   
1307287        0.0        0.0        0.0        0.0        0.0        1.0   
1307296        0.0        0.0        0.0        0.0        0.0        0.0   
1307312        0.0        0.0        0.0        0.0        0.0        0.0   
1307331        0.0        0.0        0.0        0.0        0.0        0.0   
1307334        0.0        0.0        0.0        1.0        0.0        0.0   
1307364        0.0        0.0        0.0        0.0        0.0        0.0   
1307365        0.0        0.0        0.0        2.0        0.0        0.0   
1307376        0.0        0.0        0.0        0.0        0.0        0.0   
1307377        0.0        0.0        0.0        0.0        0.0        2.0   
1307411        0.0        0.0        0.0        0.0        0.0        1.0   
1307412        0.0        1.0        0.0        0.0        0.0        0.0   
1307413 

In [20]:
product = []
for i in range(len(df1['Orders'])):
    product.append(df1['Orders'].columns[(df1['Orders'] == 1).iloc[i]].values.tolist())

In [22]:
product

[['Product 20', 'Product 21'],
 ['Product 11'],
 ['Product 19'],
 ['Product 31'],
 ['Product 11'],
 ['Product 11'],
 ['Product 11', 'Product 20'],
 ['Product 11'],
 ['Product 26'],
 ['Product 23', 'Product 26'],
 ['Product 15'],
 ['Product 11', 'Product 19', 'Product 20'],
 ['Product 13'],
 ['Product 12', 'product 1'],
 ['Product 11'],
 [],
 ['Product 11', 'Product 20'],
 ['Product 13'],
 ['Product 29'],
 ['Product 11'],
 ['Product 21'],
 ['Product 22'],
 ['Product 14'],
 ['Product 23'],
 ['Product 16'],
 ['Product 20', 'Product 27'],
 ['Product 11'],
 ['Product 20'],
 ['Product 18'],
 ['Product 24', 'Product 26'],
 ['Product 13', 'Product 16', 'product 1'],
 [],
 ['Product 13'],
 ['Product 29'],
 ['Product 12', 'Product 31'],
 ['Product 24'],
 ['Product 31'],
 ['Product 11', 'Product 12'],
 ['Product 14'],
 ['Product 30'],
 ['Product 14', 'product 1'],
 ['Product 20'],
 ['Product 19', 'Product 20'],
 ['Product 11'],
 ['Product 20', 'Product 26'],
 ['Product 11', 'Product 24'],
 ['Prod

In [23]:
te = TransactionEncoder()
te_ary = te.fit(product).transform(product)
product = pd.DataFrame(te_ary, columns=te.columns_)
product

,Product 10,Product 11,Product 12,Product 13,Product 14,Product 15,Product 16,Product 17,Product 18,Product 19,...,Product 23,Product 24,Product 25,Product 26,Product 27,Product 28,Product 29,Product 30,Product 31,product 1
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
6,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False


In [24]:
apriori(product, min_support=0.005, use_colnames=True)

,support,itemsets
0,0.056795,(Product 10)
1,0.129817,(Product 11)
2,0.101420,(Product 12)
3,0.064909,(Product 13)
4,0.038540,(Product 14)
5,0.024341,(Product 15)
6,0.044625,(Product 16)
7,0.010142,(Product 17)
8,0.079108,(Product 18)
9,0.036511,(Product 19)


In [25]:
association_rules(apriori(product, min_support=0.005, use_colnames=True), metric="confidence", min_threshold=0.1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Product 19),(Product 11),0.036511,0.129817,0.006085,0.166667,1.283854,0.001345,1.044219
1,(Product 11),(Product 20),0.129817,0.091278,0.030426,0.234375,2.567708,0.018577,1.186902
2,(Product 20),(Product 11),0.091278,0.129817,0.030426,0.333333,2.567708,0.018577,1.305274
3,(Product 18),(Product 12),0.079108,0.101420,0.008114,0.102564,1.011282,0.000091,1.001275
4,(Product 12),(Product 20),0.101420,0.091278,0.016227,0.160000,1.752889,0.006970,1.081812
5,(Product 20),(Product 12),0.091278,0.101420,0.016227,0.177778,1.752889,0.006970,1.092868
6,(Product 21),(Product 12),0.052738,0.101420,0.006085,0.115385,1.137692,0.000736,1.015786
7,(Product 16),(product 1),0.044625,0.048682,0.006085,0.136364,2.801136,0.003913,1.101527
8,(product 1),(Product 16),0.048682,0.044625,0.006085,0.125000,2.801136,0.003913,1.091857
9,(Product 20),(Product 18),0.091278,0.079108,0.012170,0.133333,1.685470,0.004950,1.062568


In [26]:
association_rules(apriori(product, min_support=0.005, use_colnames=True), metric="confidence", min_threshold=0.1).sort_values(by=['lift'])

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3,(Product 18),(Product 12),0.079108,0.101420,0.008114,0.102564,1.011282,0.000091,1.001275
6,(Product 21),(Product 12),0.052738,0.101420,0.006085,0.115385,1.137692,0.000736,1.015786
0,(Product 19),(Product 11),0.036511,0.129817,0.006085,0.166667,1.283854,0.001345,1.044219
10,(Product 18),(Product 20),0.079108,0.091278,0.012170,0.153846,1.685470,0.004950,1.073944
9,(Product 20),(Product 18),0.091278,0.079108,0.012170,0.133333,1.685470,0.004950,1.062568
5,(Product 20),(Product 12),0.091278,0.101420,0.016227,0.177778,1.752889,0.006970,1.092868
4,(Product 12),(Product 20),0.101420,0.091278,0.016227,0.160000,1.752889,0.006970,1.081812
13,(Product 27),(Product 20),0.028398,0.091278,0.006085,0.214286,2.347619,0.003493,1.156555
14,(Product 29),(Product 20),0.036511,0.091278,0.008114,0.222222,2.434568,0.004781,1.168357
2,(Product 20),(Product 11),0.091278,0.129817,0.030426,0.333333,2.567708,0.018577,1.305274


실습: 약 10000개의 market basket을 이용하여 market basket analysis 수행

In [39]:
import csv
data = []
with open("groceries.csv") as f:
    lis = csv.reader(f, delimiter=',')
    for _,x in enumerate(lis):              #print the list items 
        data.append(x)

In [40]:
data

[['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups'],
 ['tropical fruit', 'yogurt', 'coffee'],
 ['whole milk'],
 ['pip fruit', 'yogurt', 'cream cheese ', 'meat spreads'],
 ['other vegetables',
  'whole milk',
  'condensed milk',
  'long life bakery product'],
 ['whole milk', 'butter', 'yogurt', 'rice', 'abrasive cleaner'],
 ['rolls/buns'],
 ['other vegetables',
  'UHT-milk',
  'rolls/buns',
  'bottled beer',
  'liquor (appetizer)'],
 ['pot plants'],
 ['whole milk', 'cereals'],
 ['tropical fruit',
  'other vegetables',
  'white bread',
  'bottled water',
  'chocolate'],
 ['citrus fruit',
  'tropical fruit',
  'whole milk',
  'butter',
  'curd',
  'yogurt',
  'flour',
  'bottled water',
  'dishes'],
 ['beef'],
 ['frankfurter', 'rolls/buns', 'soda'],
 ['chicken', 'tropical fruit'],
 ['butter', 'sugar', 'fruit/vegetable juice', 'newspapers'],
 ['fruit/vegetable juice'],
 ['packaged fruit/vegetables'],
 ['chocolate'],
 ['specialty bar'],
 ['other vegetables'],
 ['butter milk